In [ ]:
import os


os.chdir("../")


os.getcwd()


'd:\\DA_workspace\\alzheimers_prediction\\src'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    PredefinedSplit,
    GridSearchCV,
)
from classifier.Mylib import myfuncs
import plotly.express as px
from sklearn import metrics
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.cluster import KMeans


In [ ]:
train_features = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/train_features.pkl"
)
train_target = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/train_target.pkl"
)
val_features = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/val_features.pkl"
)
val_target = myfuncs.load_python_object(
    "../artifacts/data_transformation_1/val_target.pkl"
)
features = pd.concat([train_features, val_features], axis=0)
target = pd.concat([train_target, val_target], axis=0)
spliter = PredefinedSplit(
    test_fold=[-1] * len(train_features) + [0] * len(val_features)
)


param_grid = {"1__n_clusters": range(5, 20 + 1, 5)}
pipeline = Pipeline(
    steps=[
        ("1", KMeans(n_init=10, random_state=42)),
        ("2", XGBClassifier()),
    ]
)
gs = GridSearchCV(
    pipeline,
    param_grid,
    cv=spliter,
    verbose=2,
)

gs.fit(features, target)

gs.best_params_


Fitting 1 folds for each of 4 candidates, totalling 4 fits
[CV] END ....................................1__n_clusters=5; total time=   0.5s
[CV] END ...................................1__n_clusters=10; total time=   1.1s
[CV] END ...................................1__n_clusters=15; total time=   1.4s
[CV] END ...................................1__n_clusters=20; total time=   1.9s


{'1__n_clusters': 15}

In [ ]:
importance["feature"].tolist()[:3]


['do_get_Genetic_Risk_Factor', 'do_Family_History', 'Age']

In [ ]:
df.shape


(74283, 25)

In [ ]:
df_train, df_test = train_test_split(
    df, stratify=df["Alzheimer’s Diagnosis"], test_size=0.1, random_state=42
)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


data_folder = "../artifacts/data_ingestion"
os.makedirs(data_folder, exist_ok=True)

myfuncs.save_python_object(os.path.join(data_folder, "raw_data.pkl"), df)
myfuncs.save_python_object(os.path.join(data_folder, "train_data.pkl"), df_train)
myfuncs.save_python_object(os.path.join(data_folder, "test_data.pkl"), df_test)
